# CS 195: Natural Language Processing
## Conversational Models

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ericmanley/f23-CS195NLP/blob/main/F7_3_ConversationalModels.ipynb)

## Reference

Hugging Face documentation on Blenderbot small: https://huggingface.co/docs/transformers/model_doc/blenderbot-small

## Reminder: Applied Exploration

The applied exploration for this fortnight will be a little different. I want everyone to get some experience fine-tuning an existing model, so this will be the task for the entire fortnight.

See the [workshop from last time](https://github.com/ericmanley/F23-CS195NLP/blob/main/F7_1_TransferLearning.ipynb)

Fine-tune an existing model with the following requirements
* Choose a different starting model - you can use any Hugging Face model, but consider starting with a general one like BART or Llama2.
* Choose a different data set - think about something that would be good to include in an application that interests you
* Evaluate how well it performed. For sequence-to-sequence model, try going back and using Rouge from Fortnight 1.

The Hugging Face NLP course has [examples of fine-tuning for many different tasks](https://huggingface.co/learn/nlp-course/chapter7/1).

In [1]:
import sys
!{sys.executable} -m pip install datasets keras tensorflow

## Before we get started: Attention Visualizations

These are all from the **Attention is all you Need** paper here: https://arxiv.org/pdf/1706.03762.pdf

This shows how much attention the word `making` gave to other words in the sequence. Different heads are shown in different hues

<div>
    <center>
        <img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/attention_vis1.png?raw=1">
    </center>
</div>
    

## Three different heads for the same sentence

<div>
    <center>
        <table>
            <tr>
                <td><img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/attention_vis2a.png?raw=1" width=350></td>
                <td><img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/attention_vis2b.png?raw=1" width=350></td>
                <td><img src="https://github.com/ericmanley/f23-CS195NLP/blob/main/images/attention_vis2c.png?raw=1" width=350></td>
            </tr>
        </table>
    </center>
</div>

## Conversational Models

Models used by chat bots are similar to other sequence-to-sequence models (summarization, translation, question answering), but they have been trained on transcripts of dialog.

## Loading up a Conversational Model

Blenderbot Small is a small variation that should be relatively fast to fine tune.

You can find other variants on the Hugging Face models repository.

In [2]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset


model_name = "facebook/blenderbot_small-90M"
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


All model checkpoint layers were used when initializing TFBlenderbotSmallForConditionalGeneration.

Some layers of TFBlenderbotSmallForConditionalGeneration were not initialized from the model checkpoint at facebook/blenderbot_small-90M and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Creating the first input

In [3]:
UTTERANCE = "My friends are cool but they eat too many carbs."
UTTERANCE

'My friends are cool but they eat too many carbs.'

### Tokenizing the input

In [4]:
inputs = tokenizer([UTTERANCE], return_tensors="tf")
inputs

{'input_ids': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=
array([[  42,  643,   46, 1430,   45,   52, 1176,  146,  177,  753, 2430,
           5]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

### Generating the model's response

In [5]:
reply_ids = model.generate(input_ids=inputs["input_ids"],attention_mask=inputs["attention_mask"])
reply_ids

KeyboardInterrupt: 

In [ ]:
decoded_reply = tokenizer.batch_decode(reply_ids, skip_special_tokens=True)[0]
decoded_reply

"what kind of carbs do they eat? i don't know much about carbs."

### Continued turns in the conversation

For dialogue, you need to pass the model the entire chat history

This model separates the chat messages with special `__start__` and `__end__` tokens to help the model figure out the flow of conversation.

Other models might use different separators like `<sep>` or just `\n`.

In [ ]:
REPLY = "I'm not sure"

NEXT_UTTERANCE = "My friends are cool but they eat too many carbs.__end__"
NEXT_UTTERANCE += "__start__what kind of carbs do they eat? i don't know much about carbs__end__ "
NEXT_UTTERANCE += "__start__"+REPLY

NEXT_UTTERANCE

"My friends are cool but they eat too many carbs.__end____start__what kind of carbs do they eat? i don't know much about carbs__end__ __start__I'm not sure"

In [ ]:
inputs = tokenizer([NEXT_UTTERANCE], return_tensors="tf")
next_reply_ids = model.generate(input_ids=inputs["input_ids"],attention_mask=inputs["attention_mask"])
tokenizer.batch_decode(next_reply_ids, skip_special_tokens=True)[0]

'they eat a lot of carbs. carbs are high in protein, fats, and fats.'

## Exercise

Write a loop that repeats this automatically. Prompt the user, add the user's input onto the conversation, get the model's reply, add it to the conversation, and so on.

Make sure that each time you generate a new response, you pass in the inputs for the entire conversation (the tokenizer should truncate it automatically.

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset

user_input = input("Human: ")
next_utterance = "__start__" + user_input + "__end__"
while user_input.lower() != "end":

    inputs = tokenizer([next_utterance], return_tensors="tf")
    next_reply_ids = model.generate(input_ids=inputs["input_ids"],attention_mask=inputs["attention_mask"])
    bot_response = tokenizer.batch_decode(next_reply_ids, skip_special_tokens=True)[0]
    print("Bot: ", bot_response)
    next_utterance = "__start__" + bot_response + "__end__"
    user_input = input("Human: ")
    next_utterance = "__start__" + user_input + "__end__"

## Training for Conversation

To train for conversation, you need data that consists of user inputs and responses.

This code is essentially the same as our original Fine-Tuning code, but we'll use it with a conversational model `"facebook/blenderbot_small-90M"` and a dataset consisting of ChatGPT transcripts.

In [10]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import tensorflow as tf
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset


model_name = "facebook/blenderbot_small-90M"
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

# I'm using the test split because it is much smaller
dataset = load_dataset("Open-Orca/SlimOrca",split="train")


All model checkpoint layers were used when initializing TFBlenderbotSmallForConditionalGeneration.

Some layers of TFBlenderbotSmallForConditionalGeneration were not initialized from the model checkpoint at facebook/blenderbot_small-90M and are newly initialized: ['final_logits_bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [11]:
# Shuffle the dataset
shuffled_dataset = dataset.shuffle(seed=42)

# Select a small sample
sample_size = 50  # Define your sample size
sample_dataset = shuffled_dataset.select(range(sample_size))

#if you want to use the entire dataset just uncomment the following
#sample_dataset = shuffled_dataset

In [12]:
sample_dataset

Dataset({
    features: ['conversations'],
    num_rows: 50
})

In [13]:
#displaying an example conversation
sample_dataset[0]

{'conversations': [{'from': 'system',
   'value': 'You are a helpful assistant, who always provide explanation. Think like you are answering to a five year old.',
   'weight': None},
  {'from': 'human',
   'value': 'Alan B. Miller Hall, location, Virginia; Alan B. Miller Hall, owner, College of William & Mary; Mason School of Business, country, United States; Alan B. Miller Hall, currentTenants, Mason School of Business\n\nWhat is sentence that verbalizes this data?',
   'weight': 0.0},
  {'from': 'gpt',
   'value': 'Alan B. Miller Hall is a building located in Virginia, United States, and is owned by the College of William & Mary. The Mason School of Business is currently the main tenant of the hall, and they are also part of the same college in the United States.',
   'weight': 1.0}]}

### Preprocessing

The preprocessing step is the biggest difference

In this example, I'm choosing to concatenate the system and human prompts with the GPT output as the target

In [14]:
def preprocess_function(example):
    input_texts = []
    target_texts = []

    for curr_conv in example['conversations']:

        prompt = ""

        for idx in range(len(curr_conv)-1):
            prompt += curr_conv[idx]["from"] + " "  #should be either "system" or "human" - theoretically could be an earlier "gpt" if there is more than one gpt response
            prompt += curr_conv[idx]["value"] + " " #associated prompt

        response = curr_conv[-1]["value"] #should be the gpt response

        input_texts.append(prompt)
        target_texts.append(response)

    # Tokenize inputs and targets
    model_inputs = tokenizer(input_texts, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(target_texts, max_length=512, truncation=True, padding='max_length')
    #move the target tokens into the model_inputs as the "decoder_input_ids"
    model_inputs["decoder_input_ids"] = labels["input_ids"]
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs




### Here's what one example looks like preprocessed

In [15]:
preprocess_function(sample_dataset[0:1])

{'input_ids': [[423, 15, 46, 12, 10078, 2023, 6, 73, 300, 1492, 5644, 5, 124, 71, 15, 46, 8070, 11, 12, 323, 169, 217, 5, 650, 3546, 354, 5, 3732, 775, 6, 1664, 6, 25176, 318, 337, 118, 3546, 354, 5, 3732, 775, 6, 2380, 6, 422, 10, 894, 553, 694, 332, 118, 5464, 153, 10, 455, 6, 544, 6, 247, 9326, 987, 118, 3546, 354, 5, 3732, 775, 6, 21111, 1602, 12479, 6, 5464, 153, 10, 455, 4, 44, 24, 4720, 22, 1196, 372, 27848, 36, 1419, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

### We'll use `map` to apply it to the whole dataset

In [16]:
tokenized_dataset = sample_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset

Dataset({
    features: ['conversations', 'input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
    num_rows: 50
})

In [18]:
tokenized_dataset_no_text = tokenized_dataset.remove_columns(["conversations"])
tokenized_dataset_no_text

Dataset({
    features: ['input_ids', 'attention_mask', 'decoder_input_ids', 'labels'],
    num_rows: 50
})

In [19]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_dataset_no_text,
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,
)

### Setting up the optimizer in the same way as before

The main difference here is that this model needed the SparseCategoricalCrossentropy loss function defined explicitly

In [20]:
from transformers import create_optimizer
import tensorflow as tf

num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer,loss=loss)

In [21]:
model.fit(tf_train_dataset, epochs=num_train_epochs)

Epoch 1/8
6/6 [==============================] - 74s 11s/step - loss: 10.7713
Epoch 2/8
6/6 [==============================] - 66s 11s/step - loss: 8.7370
Epoch 3/8
6/6 [==============================] - 65s 11s/step - loss: 7.2659
Epoch 4/8
6/6 [==============================] - 62s 11s/step - loss: 5.9297
Epoch 5/8
6/6 [==============================] - 62s 10s/step - loss: 4.8167
Epoch 6/8
6/6 [==============================] - 66s 11s/step - loss: 3.9749
Epoch 7/8
6/6 [==============================] - 62s 10s/step - loss: 3.4413
Epoch 8/8
6/6 [==============================] - 62s 10s/step - loss: 3.1213
